In [50]:
# -*- coding: utf-8 -*-
#from list2array2matrix2input2softmax import *
import os
import pickle
from keras.utils import to_categorical
from keras.models import load_model
import numpy as np
SRC_IP_IDX = 3-1               
DST_IP_IDX = 4-1               
SRC_PORT_IDX = 5-1             
DST_PORT_IDX = 6-1             
PROTOCOL_IDX = 7-1             
DNS_QUERY_NAME_IDX = 55-1 # domain
DNS_REQUEST_TYPE = 56-1
DNS_DOMAIN_TTL = 59-1
DNS_REPLY_IPV4IP = 60-1        
DNS_REPLY_IPV6IP = 61-1        
DNS_REPLY_RRTYPE = 62-1        
DNS_REQUEST_LEN  = 88-1        
DNS_REPLY_LENGTH = 90-1  

def pad_sequences(X, maxlen, value=0):
    S=[]
    for x in X:
        xlen = len(x)
        if xlen < maxlen:
            x.extend([value]*(maxlen-xlen))
        else:
            x = x[:maxlen]
        S.append(x)
    return S
def extract_domain(domain):
    suffix = {'.com','.la','.io', '.co', '.cn','.info', '.net', '.org','.me', '.mobi', '.us', '.biz', '.xxx', '.ca', '.co.jp', '.com.cn', '.net.cn', '.org.cn', '.mx','.tv', '.ws', '.ag', '.com.ag', '.net.ag', '.org.ag','.am','.asia', '.at', '.be', '.com.br', '.net.br', '.name', '.live', '.news', '.bz', '.tech', '.pub', '.wang', '.space', '.top', '.xin', '.social', '.date', '.site', '.red', '.studio', '.link', '.online', '.help', '.kr', '.club', '.com.bz', '.net.bz', '.cc', '.band', '.market', '.com.co', '.net.co', '.nom.co', '.lawyer', '.de', '.es', '.com.es', '.nom.es', '.org.es', '.eu', '.wiki', '.design', '.software', '.fm', '.fr', '.gs', '.in', '.co.in', '.firm.in', '.gen.in', '.ind.in', '.net.in', '.org.in', '.it', '.jobs', '.jp', '.ms', '.com.mx', '.nl','.nu','.co.nz','.net.nz', '.org.nz', '.se', '.tc', '.tk', '.tw', '.com.tw', '.idv.tw', '.org.tw', '.hk', '.co.uk', '.me.uk', '.org.uk', '.vg'}

    domain = domain.lower()
    names = domain.split(".")
    if len(names) >= 3:
        if ("."+".".join(names[-2:])) in suffix:
            return ".".join(names[-3:]), ".".join(names[:-3])
        elif ("."+names[-1]) in suffix:
            return ".".join(names[-2:]), ".".join(names[:-2])
    #print ("New domain suffix found. Use tld extract domain...")

    pos = domain.rfind("/")
    if pos >= 0: # maybe subdomain contains /, for dns tunnel tool
        ext = tldextract.extract(domain[pos+1:])
        subdomain = domain[:pos+1] + ext.subdomain
    else:
        ext = tldextract.extract(domain)
        subdomain = ext.subdomain
    if ext.suffix:
        mdomain = ext.domain + "." + ext.suffix
    else:
        mdomain = ext.domain
    return mdomain, subdomain


def iterbrowse(path):
    for home, dirs, files in os.walk(path):
        for filename in files:
            yield os.path.join(home, filename)
            
def metadata2_domain_data(log): 
    data = log.split('^')
    if not filter_metadata_dns(data):
        return None, None
    domain = data[DNS_QUERY_NAME_IDX]
    mdomain, subdomain = extract_domain(domain)
    return (mdomain, subdomain)

def filter_metadata_dns(data):
    if(len(data) < 91):
        return False

    protol  = data[PROTOCOL_IDX]
    dstport = data[DST_PORT_IDX]
    dstip   = data[DST_IP_IDX]
    qname   = data[DNS_QUERY_NAME_IDX]

    if '' == qname or '' == dstip:
        return False
    if '17' == protol and ('53' == dstport):
        return True
    return False

class LABEL(object):
    white = 0
    cdn = 1
    black = 2
def get_predict_data():
    data_path = "./xshell_data"
    black_data = []
    for path in iterbrowse(data_path):
        with open(path) as f:
            for line in f:
                mdomain, subdomain = metadata2_domain_data(line)
                if subdomain is not None:
                    black_data.append(subdomain)
    return black_data


org_X = []

def get_xshell_data():
    global org_X
    org_X = get_predict_data()
    labels = [LABEL.black]*len(org_X)

    volcab_file = "volcab.pkl"
    assert os.path.exists(volcab_file)
    pkl_file = open(volcab_file, 'rb')
    data = pickle.load(pkl_file)
    valid_chars, maxlen, max_features = data["valid_chars"], data["max_len"], data["volcab_size"]

    # Convert characters to int and pad
    X = [[valid_chars[y] if y in valid_chars else 0 for y in x] for x in org_X]
    X = pad_sequences(X, maxlen=maxlen, value=0.)

    # Convert labels to 0-1
    Y = labels
    return X, Y, maxlen, max_features


def run():
    testX, testY, max_len, volcab_size = get_xshell_data()
    print("X len:", len(testX), "Y len:", len(testY))
    print(testX[-1:])
    print(testY[-1:])
    testX=np.array(testX)
    testY=np.array(testY)
    
    """
    filename = 'dnstunnel.module'
    loaded_model =load_model(filename)

    predictions = loaded_model.predict(testX)
    print(predictions)
    cnt = 0
    global org_X
    for i,p in enumerate(predictions):
        #if abs(p[2]-testY[i][2]) < 0.1:
        if p[2]>p[1] and p[2]>p[0]:
            cnt += 1
        else:
            print("found data not detected:")
            print("original subdomain:", org_X[i])
            print("prediction compare:", p, testY[i])
    print("Dectected cnt:", cnt, "total:", len(predictions))
    print("Dectect Rate is:", cnt/(len(predictions)+.0))
    """
    filename = 'dnstunnel.module'
    loaded_model =load_model(filename)

    predictions = loaded_model.predict(testX)
    #print(predictions)
    cnt = 0
    global org_X
    for i,p in enumerate(predictions):
        #if abs(p[2]-testY[i][2]) < 0.1:
        if p[2]>p[1] and p[2]>p[0]:
            cnt += 1
        else:
            print("found data not detected:")
            print("original subdomain:", org_X[i])
            print("prediction compare:", p, testY[i])
    print("Dectected cnt:", cnt, "total:", len(predictions))
    print("Dectect Rate is:", cnt/(len(predictions)+.0))

if __name__ == "__main__":
    run()


X len: 1180 Y len: 1180
[[9, 15, 36, 15, 36, 34, 31, 27, 27, 26, 29, 37, 23, 25, 34, 15, 31, 36, 33, 34, 17, 38, 23, 23, 11, 37, 20, 34, 1, 15, 36, 38, 23, 33, 17, 7, 25, 8, 17, 15, 17, 23, 25, 27, 36, 33, 25, 6, 9, 42, 23, 26, 37, 25, 1, 13, 1, 38, 21, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0